In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [1]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points}

**Please be aware that this homework assignment requires installation of several packages in your course environment. It's possible that you'll encounter installation challenges, which might be frustrating. However, remember that solving these issues is not wasting time but it is an essential skill for anyone aspiring to work in data science or machine learning.**

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W1/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb. If the pdf or html also fail to render on Gradescope, please create two files for your homework: hw6a.ipynb with Exercise 1 and hw6b.ipynb with Exercises 2 and 3 and submit these two files in your submission.  
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [2]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [3]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [4]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [7]:
word1 = "burgers"

glove_wiki_vectors.most_similar(word1)

[('hamburgers', 0.8610642552375793),
 ('sandwiches', 0.781425952911377),
 ('fries', 0.7542682886123657),
 ('steaks', 0.7456540465354919),
 ('steak', 0.7047969102859497),
 ('burger', 0.7014811038970947),
 ('cheeseburgers', 0.6995593309402466),
 ('chicken', 0.6964924335479736),
 ('salads', 0.6875000596046448),
 ('fried', 0.6868302226066589)]

In [8]:
word2 = "house"
glove_wiki_vectors.most_similar(word2)

[('office', 0.7581613659858704),
 ('senate', 0.7204986214637756),
 ('room', 0.7149736285209656),
 ('houses', 0.6888046264648438),
 ('capitol', 0.6851760745048523),
 ('building', 0.6847286224365234),
 ('home', 0.672031044960022),
 ('clinton', 0.6707027554512024),
 ('congressional', 0.6692575812339783),
 ('mansion', 0.665092408657074)]

In [9]:
word3 = "shark"
glove_wiki_vectors.most_similar(word3)

[('whale', 0.784017026424408),
 ('dolphin', 0.7335217595100403),
 ('crocodile', 0.6806946396827698),
 ('fish', 0.6578195095062256),
 ('sharks', 0.6473532915115356),
 ('turtle', 0.6445284485816956),
 ('monkey', 0.625479519367218),
 ('elephant', 0.6209661364555359),
 ('swordfish', 0.619266927242279),
 ('alligator', 0.6062195301055908)]

In [10]:
word4 = "panda"
glove_wiki_vectors.most_similar(word4)

[('pandas', 0.7574485540390015),
 ('cub', 0.7197702527046204),
 ('zoo', 0.6617313027381897),
 ('wolong', 0.6480264067649841),
 ('elephant', 0.5793296098709106),
 ('dolphin', 0.5778335928916931),
 ('gorilla', 0.5638635158538818),
 ('monkey', 0.5408830642700195),
 ('bird', 0.5377456545829773),
 ('cat', 0.5304447412490845)]

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [11]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [14]:
for word_1, word_2 in word_pairs:
    print(f"The cosine similarity between {word_1} and {word_2} is {glove_wiki_vectors.similarity(word_1,word_2)}")



The cosine similarity between coast and shore is 0.7000270485877991
The cosine similarity between clothes and closet is 0.5462760329246521
The cosine similarity between old and new is 0.6432486772537231
The cosine similarity between smart and intelligent is 0.7552730441093445
The cosine similarity between dog and cat is 0.8798073530197144
The cosine similarity between tree and lawyer is 0.07671946287155151


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [19]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [20]:
for i in test_words:
    if i in glove_wiki_vectors:
        print(f"there is a representation for the word'{i}'")
    else:
        print(f"there is NOT a representation for the word'{i}'")

there is NOT a representation for the word'covididiot'
there is NOT a representation for the word'fomo'
there is a representation for the word'frenemies'
there is NOT a representation for the word'anthropause'
there is NOT a representation for the word'photobomb'
there is NOT a representation for the word'selfie'
there is NOT a representation for the word'pxg'
there is NOT a representation for the word'pacg'
there is a representation for the word'cct'
there is a representation for the word'escc'


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [21]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [22]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [23]:
glove_wiki_vectors.similarity("aboriginal", "success")

0.14283241

In [24]:
glove_wiki_vectors.similarity("white", "success")

0.3518238

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [32]:
analogy("man", "football", "woman")


man : football :: woman : ?


,Analogy word,Score
0,basketball,0.787933
1,soccer,0.745967
2,hockey,0.693960
3,volleyball,0.685398
4,softball,0.671104
5,professional,0.659303
6,collegiate,0.655488
7,rugby,0.654050
8,league,0.650230
9,baseball,0.637469


In [36]:
analogy("woman", "shopping", "man")

woman : shopping :: man : ?


,Analogy word,Score
0,mall,0.695505
1,malls,0.638853
2,store,0.622103
3,home,0.606814
4,shop,0.598452
5,busy,0.594252
6,holiday,0.591183
7,parking,0.590809
8,hotel,0.586590
9,attractions,0.578486


In [28]:
print(glove_wiki_vectors.similarity("man", "strong"))
print(glove_wiki_vectors.similarity("woman", "strong"))

0.47332558
0.3862985


In [29]:
print(glove_wiki_vectors.similarity("man", "independence"))
print(glove_wiki_vectors.similarity("woman", "independence"))

0.28878838
0.2746406


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

### my examples

For the man:football::woman, there does not seem to be a bias, all the outputs have high levels of correlation and are all sportys related.


For the woman:shopping::man, there does not seem to be a bias, the ouputs are mostly shopping related, however they are mostly buildings which makes me think the embedding did not quite catch the correlations properly.

For the third one, there is a higher similarity between man and strong than woman and strong, which has a lot of bias and can be a source of harm in real life.

For the final woman there is a similar similarity between woman and independence and man and independence, which means there is no apparent bias here.

### cause real harm in the world

If an AI model is trained on embeddings to analyze crime reports in a city to determine policing, these biases in the embeddings can cause overpolicing of minority communities.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [37]:
from sklearn.datasets import fetch_20newsgroups

In [38]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [39]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [44]:
import spacy
import re
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [105]:
def preprocess(text):

    irrelevant=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE", "VERB" ]
    
    
    text = re.sub(r"https?://\S+|www\.\S+", "URL", text)
    text = re.sub(r"\S+@\S+\.\S+", " ", text)
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"[\r\n\t]+", " ", text) 
    text = re.sub(r"[/\\]", " ", text)
    text = re.sub(r"[^\w\s]", "", text)
    
    doc = nlp(text)
    clean_text = []

  
    for token in doc:
        if (
            not token.is_stop  
            and len(token) > 2  
            and token.pos_ not in irrelevant  
            and not token.like_url  
            and not token.like_email 
            and not token.like_num  
        ):
            lemma = token.lemma_ 
            clean_text.append(lemma.lower())  
    

    return " ".join(clean_text)

In [106]:
df["text_pp"] = df["text"].apply(preprocess)

In [107]:
df

,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,usc sence usc paragraph shall unlawful person ...
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref algorithm bit hard point plan...
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand idea newsgroup aspect graphic p...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,problem help window child event_mask buttonpre...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...
...,...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics,software shadow simple raytracer visualization...
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x,archivename xfaq subject patch release new pub...
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns,nice question difference fed literacy fed lite...
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x,gateway correct xconfig file entry graphic car...


In [108]:
df.iloc[2:6]

,text,target,target_name,text_pp
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand idea newsgroup aspect graphic p...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,problem help window child event_mask buttonpre...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...
5,"Hi,\n I'd like to subscribe to Leadership Ma...",5,soc.religion.christian,like leadership magazine disk paper disk illus...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points}

**Your tasks:**

- Outline the preprocessing steps you carried out in the previous exercise (bullet point format is fine), providing a brief justification when necessary. 

> You might want to wait to answer this question till you are done with Exercise 2 and you have finalized the preprocessing steps in 2.1. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

URL: 

Not useful for finding a topic for a text and could mislead the model. Replaced them with "URL" .


Email Address:

Not likely to contribute positively for topic modeling as they mostly contain the user's name. Replaced with a generic "EMAIL".


Numbers:

Numbers don't havie any semantic meaning in for topic modelling, so I completely removed them.


Tabs:

Not very useful for topic modeling, because they don't convey much more meaning than a normal space.


Slashes:

not informative for topic modeling.


Punctuation:

not helpful in topic modeling and may introduce noise, their presence or there lack of, does not change the meaning of the words themselves. 

Lemmatization:

lemmatization reduces words to their root form, which can improve the quality of topic modeling because we will have less total tokens.


Stopwords:

like ("and" and "the") do not carry meaningful information for topic modeling.

Verbs:
I also removed verbs, because I felt like verbs like thinking and taking can apply to so many topic and are non-specific so it make mislead the model

Other categories in the "irelevant" generally do not provide any semantic meaning that helps differentiate topics.



<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

I think that the one with the 4 components might work the best, because it distinguishes between computers, sport, politics(guns and middle east), and God(athiesm and chritianity).

In [109]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

In [110]:


vec = CountVectorizer()

df_vec= vec.fit_transform(df["text_pp"])
df_vec

<4563x35401 sparse matrix of type '<class 'numpy.int64'>'
	with 196593 stored elements in Compressed Sparse Row format>

In [111]:
n_topics = 8
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=42)
lda.fit(df_vec) 

sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())
import mglearn

mglearn.tools.print_topics(
    topics=range(n_topics),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
game          body          file          god           people        
team          dead          program       people        israel        
year          compass       image         jesus         time          
good          group         window        thing         israeli       
player        int           available     time          right         
season        town          server        good          woman         
time          april         entry         way           child         
hockey        azeri         version       church        thing         
league        child         system        question      way           
period        helicopter    line          life          armenians     


topic 5       topic 6       topic 7       
--------      --------      --------      
widget        turkish       gun           
lib           arm

In [112]:
n_topics = 6
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=42)
lda.fit(df_vec) 

sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())
import mglearn

mglearn.tools.print_topics(
    topics=range(n_topics),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
game          file          file          god           people        
team          gun           program       people        gun           
year          firearm       image         jesus         armenian      
good          bill          window        thing         armenians     
player        weapon        available     time          israel        
season        control       entry         good          government    
time          section       server        way           turkish       
hockey        police        version       question      right         
league        congress      system        church        time          
new           article       information   life          year          


topic 5       
--------      
lib           
widget        
value         
window        
event         
bos           
point         
application

In [113]:
n_topics = 4
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=42)
lda.fit(df_vec) 

sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())
import mglearn

mglearn.tools.print_topics(
    topics=range(n_topics),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
game          gun           file          god           
team          armenian      program       people        
year          people        image         time          
good          armenians     window        thing         
player        turkish       available     jesus         
time          weapon        server        good          
season        government    entry         way           
hockey        file          version       question      
league        firearm       widget        jews          
new           crime         system        israel        




In [114]:
n_topics = 2
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=42)
lda.fit(df_vec) 

sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())
import mglearn

mglearn.tools.print_topics(
    topics=range(n_topics),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       
--------      --------      
game          file          
year          people        
team          god           
people        time          
good          program       
time          good          
armenian      thing         
player        image         
armenians     way           
turkish       gun           




<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

In [115]:
n_topics = 4
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=42)
lda.fit(df_vec) 

sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())
import mglearn

mglearn.tools.print_topics(
    topics=range(n_topics),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
game          gun           file          god           
team          armenian      program       people        
year          people        image         time          
good          armenians     window        thing         
player        turkish       available     jesus         
time          weapon        server        good          
season        government    entry         way           
hockey        file          version       question      
league        firearm       widget        jews          
new           crime         system        israel        




topic0 = sports

topic1 = politics

topic2= computers

topic3= God

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [ ]:
topic_distributions = lda.transform(df_vec)

print(f"Document {1}: {topic_distributions[0]}")
print(df['text'].iloc[0])

Document 1: [0.008173   0.72118851 0.00744308 0.26319542]
You know, I was reading 18 U.S.C. 922 and something just did not make 
sence and I was wondering if someone could help me out.

Say U.S.C. 922 :

(1) Except as provided in paragraph (2), it shall be unlawful for
any person to transfer or possess a machinegun.

 Well I got to looking in my law dictionary and I found that a "person" 
might also be an artificial entity that is created by government 
and has no rights under the federal constitution. So, what I 
don't understand is how a statute like 922 can be enforced on 
an individual. So someone tell me how my government can tell
me what I can or cannot possess. Just passing a law 
does not make it LAW. Everyone knows that laws are constitional
until it goes to court. So, has it ever gone to court, not
just your run of the mill "Ok I had it I am guilty, put me in jail"

Has anyone ever claimed that they had a right to possess and was told
by the Supreme Court that they didn't hav

# The model says topic 1, which is politics

I would also categorize this as politics, it seems to be about gun laws 

In [127]:

print(f"Document {2}: {topic_distributions[1]}")
print(df['text'].iloc[1])

#computers

Document 2: [0.00463567 0.00456023 0.79867598 0.19212811]



It's not a bad question: I don't have any refs that list this algorithm
either. But thinking about it a bit, it shouldn't be too hard.

1) Take three of the points and find the plane they define as well as
the circle that they lie on (you say you have this algorithm already)

2) Find the center  of this circle. The line passing through this center
perpendicular to the plane of the three points passes through the center of
the sphere.

3) Repeat with the unused point and two of the original points. This
gives you two different lines that both pass through the sphere's
origin. Their interection is the center of the sphere.

4) the radius is easy to compute, it's just the distance from the center to
any of the original points.

I'll leave the math to you, but this is a workable algorithm. :-)


An alternate method would be to take pairs of points: the plane formed
by the perpendicular bisector of each line segment pair also cont

# The model says topic 2, which is computers

I would also categorize this as computers, it seems to be about an algorithm

In [128]:
#god

print(f"Document {3}: {topic_distributions[2]}")
print(df['text'].iloc[2])

Document 3: [0.00689753 0.00673755 0.2828812  0.70348372]

Actuallay I don't, but on the other hand I don't support the idea of having
one newsgroup for every aspect of graphics programming as proposed by Brian,
in his reply to my original posting.
I would suggest a looser structure more like a comp.graphics.programmer,
comp.graphics.hw_and_sw
The reason for making as few groups as possible is for the same reason you
say we shouldn't spilt up, not to get to few postings every day.
I takes to much time to browse through all postings just to find two or 
three I'm interested in.

I understand and agree when you say you want all aspects of graphics in one
meeting. I agree to some extension. I see news as a forum to exchange ideas,
help others or to be helped. I think this is difficult to achive if there
are so many different things in one meeting.

Good evening netters|-)


# The model says topic 3, which is god

I would categorize this as computers,as it seems to be about programming. The model got this one wrong.

In [129]:
#computers

print(f"Document {4}: {topic_distributions[3]}")
print(df['text'].iloc[3])

Document 4: [0.43645542 0.06600913 0.48329365 0.0142418 ]
The following problem is really bugging me,
and I would appreciate any help.

I create two windows:

w1 (child to root) with event_mask = ButtonPressMask|KeyPressMask;
w2 (child to w1) with do_not_propagate_mask = ButtonPressMask|KeyPressMask;


Keypress events in w2 are discarded, but ButtonPress events fall through
to w1, with subwindow set to w2.

FYI, I'm using xnews/olvwm.

Am I doing something fundamentally wrong here?


# The model says topic 2, which is computers

I would categorize this as computers,as it seems to have code in it. The model got this one right.

In [130]:
#politics
print(f"Document {5}: {topic_distributions[4]}")
print(df['text'].iloc[4])

Document 5: [0.00769301 0.79451093 0.0083276  0.18946846]


  This is the latest from UPI 

     Foreign Ministry spokesman Ferhat Ataman told journalists Turkey was
     closing its air space to all flights to and from Armenia and would
     prevent humanitarian aid from reaching the republic overland across
     Turkish territory.

  
   Historically even the most uncivilized of peoples have exhibited 
   signs of compassion by allowing humanitarian aid to reach civilian
   populations. Even the Nazis did this much.

   It seems as though from now on Turkey will publicly pronounce 
   themselves 'hypocrites' should they choose to continue their
   condemnation of the Serbians.



--


# The model says topic 1, which is politics

I would categorize this as politics as well,as it seems to be about Turkey.

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

1. Recommender systems suggest items similar to those a user has interacted with, based on various attributes of an item. If the msystem is recommending movies, then it would be looking at genre, actors, themes and seeing how different users respond to those attributes and making recommendations based on that.

2. 
Filter Bubbles: Users are only exposed to similar content over and over which can cause things like bias enforcement, where instead of people being exposed to multifaceted ideas, they are echoed with whatever they already believe.

Ethics: Because of the fact that Recommender systen are trained with embeddings, they may perpetuate biases in recommendations.In addition, user data is stored and tracked, and this information can be consequential in the wrong hands.


3. It involves taking a pre-trained model and tuning it for our specific purposes. In this scenario, the model would have already learned the language semantics that can be applied to a related task. It is much more efficient than training a model from scratch.

<!-- END QUESTION -->

<br><br><br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)